# MergeSort problem:
Given a array with random numbers, sort them. Using merge sort algorithm, the time complexity can be effectively reduced to O(nlogn). For example, given an array [9, 2, 3, 5, 1, -1, 0, 99], the sorted array from minimal to maximal should be returned.

In [30]:
def merge_sort(arr):
    # stop the recursion when divided into single element
    if len(arr) <= 1:
        return arr
    #split the whole array into two sub-branches
    split_idx = len(arr) // 2
    left_tree = arr[:split_idx]
    right_tree = arr[split_idx:]
    #recursive split the array until one element is found
    merge_sort(left_tree)
    merge_sort(right_tree)
    #i,j,k are the indices of the elements in each array
    i, j, k = 0, 0, 0
    while i < len(left_tree) and j < len(right_tree):
        if left_tree[i] < right_tree[j]:
            arr[k] = left_tree[i]
            i += 1
        else:
            arr[k] = right_tree[j]
            j += 1
        k += 1
    #the leftover should be appended to the array
    while i < len(left_tree):
        arr[k] = left_tree[i]
        i += 1
        k += 1
    while j < len(right_tree):
        arr[k] = right_tree[j]
        j += 1
        k += 1

    return arr

arr = [9, 2, 3, 5, 1, -1, 0, 99]
print(merge_sort(arr))

[-1, 0, 1, 2, 3, 5, 9, 99]


## Comments:
To better explain the divide and conquer process, here I attached two pictures (pls bear with my doggy handwritting, xd)
<!-- ![p1](./pictures/mergesortp2.png) -->
<img src="./pictures/mergesortp2.png" alt="p2" width="600" height="500"/>
<img src="./pictures/mergesortp1.png" alt="p1" width="700" height="500"/>


# Inverse Count problem
The inverse count problem can be solved in complexity O(nlogn) compared to normal brute-force in O(n^2).
Count of smaller elements on right side of each element in an Array using Merge sort.

For example: 具体来说，如果我们有一个序列 S，那么两个元素 S[i] 和 S[j] 形成一个逆序，如果满足以下两个条件：
i < j
S[i] > S[j]
例如，在序列 [3, 1, 2, 6, 5, 4] 中，存在五个这样的逆序：(3, 1), (3, 2), (6, 5), (6, 4), (5, 4)。

逆序问题的重要性：
逆序的数量是衡量序列“有序程度”的一个指标。在某些应用中，它可以用来衡量列表或数组需要多少交换才能完全排序。这在计算复杂性、数据分析、计算机图形学等领域都有应用。


In [42]:
#brute-force method:
def brute_force(arr):
    count = 0
    record_revert_nums = []
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            if arr[i] > arr[j]:
                count += 1
                record_revert_nums.append((arr[i], arr[j]))

    return count, record_revert_nums

arr = [3, 1, 2, 6, 5, 4]
print(brute_force(arr))

(5, [(3, 1), (3, 2), (6, 5), (6, 4), (5, 4)])


Notice, the brute-force algorithm has two loops, leading to O(n^2). To reduce the time complexity for huge input, we now introduce the merge-sort algorithm below. The idea is similar: taking the example from the handwritting above and we slightly change the array to [9, 2, 5, 3]. Now, they have inverses of (9, 2/3/5) and (5, 3). When it splits to the minimal elements, the first comparision during the recursive are the pairs (9, 2) and (5, 3), so the reverse count returns 2. Then, one layer above, the trees will be sorted pairs [2, 9] and [3, 5]. The 'while' loop compares: 2 vs. 3 (good, i+1=1, j=0); 9 vs. 3 (reversed, i=1, j+1=1, reverse_count+1); 9 vs. 5 (reversed, i=1, j+1=2, reverse_count+1).

In [37]:
#merge-sort algorithm:
def merge_sort_revert(arr):
    # No inversions in a single-element array
    if len(arr) <= 1:
        return arr, 0, []
    #split the whole array into two sub-branches
    split_idx = len(arr) // 2
    left_tree = arr[:split_idx]
    right_tree = arr[split_idx:]
    #recursive split the array until one element is found
    left_tree, left_inversions, left_records = merge_sort_revert(left_tree)
    right_tree, right_inversions, right_records = merge_sort_revert(right_tree)
    inversions = left_inversions + right_inversions

    # Combine the inversion records from the left and right
    record_inversion_nums = []
    record_inversion_nums.extend(left_records)
    record_inversion_nums.extend(right_records)

    #i,j,k are the indices of the elements in each array
    i, j, k = 0, 0, 0
    while i < len(left_tree) and j < len(right_tree):
        if left_tree[i] < right_tree[j]:
            arr[k] = left_tree[i]
            i += 1
        else:
            arr[k] = right_tree[j]
            #all the numbers remained in the left-tree will be larger than the right half, leading to inversions
            inversions += (len(left_tree) - i)
            #to record the inverted numbers
            for index in range(i, len(left_tree)):
                record_inversion_nums.append((left_tree[index], right_tree[j]))
            j += 1
        k += 1
    #the leftover should be appended to the array
    while i < len(left_tree):
        arr[k] = left_tree[i]
        i += 1
        k += 1
    while j < len(right_tree):
        arr[k] = right_tree[j]
        j += 1
        k += 1

    return arr, inversions, record_inversion_nums

arr = [3, 1, 2, 6, 5, 4]
merge_sort_revert(arr)

([1, 2, 3, 4, 5, 6], 5, [(3, 1), (3, 2), (5, 4), (6, 4), (6, 5)])

Comment:
In the context of this recursive function, record_inversion_nums is re-initialized to an empty list [] at the beginning of each function call (i.e., each recursion level). This is because each function call is dealing with a different subsection of the original array, and we start with a "clean slate" for recording inversions specific to that subsection.

However, the inversions recorded in deeper levels of recursion (i.e., in the recursive calls that sort the left and right halves of the current subsection) are not lost. These are returned by the recursive calls as left_records and right_records.

# Cloest Pair problem
The "closest pair problem" is a common challenge in computer science and geometry, often utilized in fields like computational geometry. The problem involves finding the two closest points among a set of given points in a plane based on Euclidean distance.

## Overview
Input: You are given a set 'P' containing 'n' points, P = {p1, p2, ..., pn}, each with coordinates in a two-dimensional space.

Objective: Find the pair of points (pi, pj) that have the smallest Euclidean distance among all possible pairs of points. The Euclidean distance between two points (x1, y1) and (x2, y2) is calculated as sqrt((x1 - x2)² + (y1 - y2)²).

Solving this problem in a brute-force manner would require calculating the distance between every pair of points and then finding the minimum, leading to an O(n^2) complexity. However, there's a more efficient approach using a technique similar to MergeSort, bringing down the complexity to O(n (log n)^2) or even O(n log n) with more sophisticated implementations. Below is a simplified explanation of how it's done. Following the above habit, I will use the brute-force first and then optimise it.

In [54]:
#brute-force algorithm
import math
def brute_force_cloest(points):
    min_distance = math.inf
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            distance = math.sqrt((points[i][0]-points[j][0])**2+(points[i][1]-points[j][1])**2)
            if  distance < min_distance:
                min_distance = distance
                closest_pair = ([points[i], points[j]])
    return min_distance, closest_pair
            

points = [(1,3), (10,17), (1,4), (18,24), [10.5,17]]
brute_force_cloest(points)

(0.5, [(10, 17), [10.5, 17]])

The brute-force method always be intuitive and straightforward. I keep practicing this for getting myself familar with the coding and it usually enhances our confidence when starting from an easy approach. Next, let's discuss the merge-sort method:

1